[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io/)

# Machine Learning Methods

## Supervised Learning - Regression - Linear Regression with ${L}_{1}$ / ${L}_{2}$ Regularization

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 0.1.000 | 12/02/2023 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/MachineLearningMethods/2023_01/0024RegressorRegularization.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning
from sklearn.linear_model import Lasso, Ridge
from sklearn.linear_model import lars_path, Lasso, lasso_path
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

# Miscellaneous
import os
from platform import python_version
import random

# Typing
from typing import Tuple

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter
from IPython import get_ipython
from IPython.display import Image, display
from ipywidgets import Dropdown, FloatSlider, interact, IntSlider, Layout

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

In [ ]:
# Configuration
# %matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants

FIG_SIZE_DEF    = (8, 8)
ELM_SIZE_DEF    = 50
CLASS_COLOR     = ('b', 'r')
EDGE_COLOR      = 'k'
MARKER_SIZE_DEF = 10
LINE_WIDTH_DEF  = 2


In [ ]:
# Fixel Algorithms Packages


## Linear Regression with LASSO (${L}_{1}$) / Ridge (${L}_{2}$) Regularization

Regularization is a simple and effective way to balance the adaptation of the model to the data.  
It allows us to use complex model yet to tune it to prevent overfit.

The models of optimization:

 - LASSO: $\arg \min_{\boldsymbol{w}} \frac{1}{2} {\left\| X \boldsymbol{w} - \boldsymbol{y} \right\|}_{2}^{2} + \lambda {\left\| \boldsymbol{w} \right\|}_{1}$.  
   Promotes sparsity, modeled by a Laplace prior of the coefficients of $\boldsymbol{w}$.
 - Ridge: $\arg \min_{\boldsymbol{w}} \frac{1}{2} {\left\| X \boldsymbol{w} - \boldsymbol{y} \right\|}_{2}^{2} + \lambda {\left\| \boldsymbol{w} \right\|}_{2}^{2}$.  
   Promotes damped coefficients, modeled by a Gaussian prior of the coefficients of $\boldsymbol{w}$.

* <font color='brown'>(**#**)</font> The LASSO model is implemented by the `Lasso` class in SciKit Learn.
* <font color='brown'>(**#**)</font> The Ridge model is implemented by the `Ridge` class in SciKit Learn.
* <font color='brown'>(**#**)</font> Both `Lasso` and `Ridge` (As well as `LinearRegression`) have the `positive` parameter to enforce non negative values for $\boldsymbol{w}$.
* <font color='brown'>(**#**)</font> The SciKit Learn classes use $\alpha$ instead of $\lambda$ in their optimization function.

In this notebook we'll show the effect of the regularization on the fitness level of the model.

In [ ]:
# Parameters

# Data Generation
numSamples  = 30
noiseStd    = 17.5

vP = np.array([0.5, 0.25, 2, 5])

# Model
lPolyDeg = [3, 5, 7, 9]
lλ       = list(np.linspace(0, 25, 500)) #<! Pay attention that for λ = 0 it is better to use LinearRegression model (Numerical reasons)
numSamplesTrain = 20
numSamplesTest  = numSamples - numSamplesTrain

# Data Visualization
gridNoiseStd = 0.05
numGridPts   = 500

In [ ]:
# Auxiliary Functions

def PlotRegressionData( mX: np.ndarray, vY: np.ndarray, hA:plt.Axes = None, figSize: Tuple[int, int] = FIG_SIZE_DEF, elmSize: int = ELM_SIZE_DEF, classColor: Tuple[str, str] = CLASS_COLOR, axisTitle: str = None ) -> plt.Axes:

    if hA is None:
        hF, hA = plt.subplots(figsize = figSize)
    else:
        hF = hA.get_figure()
    
    if np.ndim(mX) == 1:
        mX = np.reshape(mX, (mX.size, 1))

    numSamples = len(vY)
    numDim     = mX.shape[1]
    if (numDim > 2):
        raise ValueError(f'The features data must have at most 2 dimensions')
    
    # Work on 1D, Add support for 2D when needed
    # See https://matplotlib.org/stable/api/toolkits/mplot3d.html
    hA.scatter(mX[:, 0], vY, s = elmSize, color = classColor[0], edgecolor = 'k', label = f'Samples')
    hA.axvline(x = 0, color = 'k')
    hA.axhline(y = 0, color = 'k')
    hA.set_xlabel('${x}_{1}$')
    # hA.axis('equal')
    if axisTitle is not None:
        hA.set_title(axisTitle)
    hA.legend()
    
    return hA

def PlotPolyFit( vX: np.ndarray, vY: np.ndarray, vP: np.ndarray = None, P: int = 1, numGridPts: int = 1001, hA:plt.Axes = None, figSize: Tuple[int, int] = FIG_SIZE_DEF, markerSize: int = MARKER_SIZE_DEF, lineWidth: int = LINE_WIDTH_DEF, axisTitle: str = None ):

    if hA is None:
        hF, hA = plt.subplots(1, 2, figsize = figSize)
    else:
        hF = hA[0].get_figure()

    numSamples = len(vY)

    # Polyfit
    vW    = np.polyfit(vX, vY, P)
    
    # MSE
    vHatY = np.polyval(vW, vX)
    MSE   = (np.linalg.norm(vY - vHatY) ** 2) / numSamples
    
    # Plot
    xx  = np.linspace(np.floor(np.min(vX)), np.ceil(np.max(vX)), numGridPts)
    yy  = np.polyval(vW, xx)

    hA[0].plot(vX, vY, '.r', ms = 10, label = '$y_i$')
    hA[0].plot(xx, yy, 'b',  lw = 2,  label = '$\hat{f}(x)$')
    hA[0].set_title (f'$P = {P}$\nMSE = {MSE}')
    hA[0].set_xlabel('$x$')
    # hA[0].axis(lAxis)
    hA[0].grid()
    hA[0].legend()
    
    hA[1].stem(vW[::-1], label = 'Estimated')
    if vP is not None:
        hA[1].stem(vP[::-1], linefmt = 'C1:', markerfmt = 'D', label = 'Ground Truth')
    numTicks = len(vW) if vP is None else max(len(vW), len(vP))
    hA[1].set_xticks(range(numTicks))
    hA[1].set_title('Coefficients')
    hA[1].set_xlabel('$w$')
    hA[1].legend()

    # return hA


## Generate / Load Data

In the following we'll generate data according to the following model:

$$ y_{i} = f \left( x_{i} \right) + \epsilon_{i} $$

Where

$$ f \left( x \right) = \frac{1}{2} {x}^{3} + \frac{1}{4} {x}^{2} + 2 x + 5 $$


In [ ]:
# Defining the Generating Function
def f( vX: np.ndarray, vP: np.ndarray ):
    
    return np.polyval(vP, vX)


hF = lambda vX: f(vX, vP)

In [ ]:
# Loading / Generating Data

vX = np.linspace(-5, 5, numSamples, endpoint = True) + (gridNoiseStd * np.random.randn(numSamples))
vN = noiseStd * np.random.randn(numSamples)
vY = hF(vX) + vN

print(f'The features data shape: {vX.shape}')
print(f'The labels data shape: {vY.shape}')

### Plot Data

In [ ]:
# Display the Data

PlotRegressionData(vX, vY)

plt.show()

## Train Test Split

In order to show the importance of the regularization, we'll apply a split on the data.

In [ ]:
# Train Test Split

mX = np.reshape(vX, (-1, 1))
mXTrain, mXTest, vYTrain, vYTest = train_test_split(mX, vY, test_size = numSamplesTest, train_size = numSamplesTrain, shuffle = True)

* <font color='red'>(**?**)</font> What does `stratify` mean in the context of splitting data for regression? 

## Train a Regularized Polyfit Regressor

The regularized PolyFit optimization problem is given by:

$$ \arg \min_{\boldsymbol{w}} {\left\| \boldsymbol{X} \boldsymbol{w} - \boldsymbol{y} \right|}_{2}^{2} + \lambda R \left( \boldsymbol{w} \right) $$

Where

$$
\boldsymbol{X} = \begin{bmatrix} 1 & x_{1} & x_{1}^{2} & \cdots & x_{1}^{p} \\
1 & x_{2} & x_{2}^{2} & \cdots & x_{2}^{p} \\
\vdots & \vdots & \vdots &  & \vdots \\
1 & x_{N} & x_{N}^{2} & \cdots & x_{N}^{p}
\end{bmatrix}
$$

And $R \left( \boldsymbol{w} \right)$ is the regularization function.


In [ ]:
# Pipelines

# We could use the same instance of polynomial features object for both
pPolyFitL1 = Pipeline([('PolyFeatures', PolynomialFeatures(include_bias = False)), ('Regressor', Lasso(fit_intercept = True))])
pPolyFitL2 = Pipeline([('PolyFeatures', PolynomialFeatures(include_bias = False)), ('Regressor', Ridge(fit_intercept = True))])


In [ ]:
# Score Data Frame

numComb = len(lPolyDeg) * len(lλ)
dData   = {'Poly Degree': [], 'λ': [], 'Train R2 L1 Regularization': [0.0] * numComb, 'Test R2 L1 Regularization': [0.0] * numComb, 'Train R2 L2 Regularization': [0.0] * numComb, 'Test R2 L2 Regularization': [0.0] * numComb}

for ii, polyDeg in enumerate(lPolyDeg):
    for jj, paramλ in enumerate(lλ):
        dData['Poly Degree'].append(polyDeg)
        dData['λ'].append(paramλ)
#===============================================================#

dfModelScore = pd.DataFrame(data = dData)
dfModelScore

In [ ]:
# Scoring the Models

for ii in range(numComb):
    polyDeg  = dfModelScore.loc[ii, 'Poly Degree']
    paramλ   = dfModelScore.loc[ii, 'λ']

    print(f'Processing model {ii + 1:03d} out of {numComb} with `degree` = {polyDeg} and `λ` = {paramλ}.')

    # The `__` Trick for Pipelines
    pPolyFitL1.set_params(PolyFeatures__degree = polyDeg)
    pPolyFitL1.set_params(Regressor__alpha = paramλ)
    pPolyFitL2.set_params(PolyFeatures__degree = polyDeg)
    pPolyFitL2.set_params(Regressor__alpha = paramλ)

    pPolyFitL1.fit(mXTrain, vYTrain)
    pPolyFitL2.fit(mXTrain, vYTrain)

    dfModelScore.loc[ii, 'Train R2 L1 Regularization'] = pPolyFitL1.score(mXTrain, vYTrain)
    dfModelScore.loc[ii, 'Test R2 L1 Regularization'] = pPolyFitL1.score(mXTest, vYTest)
    dfModelScore.loc[ii, 'Train R2 L2 Regularization'] = pPolyFitL2.score(mXTrain, vYTrain)
    dfModelScore.loc[ii, 'Test R2 L2 Regularization'] = pPolyFitL2.score(mXTest, vYTest)

In [ ]:
dfModelScore

In [ ]:
numFigures = len(lPolyDeg)
numRows    = 2
numCols    = int(np.ceil(numFigures / numRows))

hF, hA = plt.subplots(nrows = numRows, ncols = numCols, figsize = (18, 10))
hA = hA.flat

for ii, ax in enumerate(hA):
    if ii >= len(lPolyDeg):
        continue
    dsIdx = dfModelScore['Poly Degree'] == lPolyDeg[ii]
    ax.plot(lλ, dfModelScore.loc[dsIdx, 'Train R2 L1 Regularization'], color = 'C0', ls = ':', label = 'Train L1')
    ax.plot(lλ, dfModelScore.loc[dsIdx, 'Test R2 L1 Regularization'], color = 'C0', ls = '-', label = 'Test L1')
    ax.plot(lλ, dfModelScore.loc[dsIdx, 'Train R2 L2 Regularization'], color = 'C1', ls = ':', label = 'Train L2')
    ax.plot(lλ, dfModelScore.loc[dsIdx, 'Test R2 L2 Regularization'], color = 'C1', ls = '-', label = 'Test L2')

    # ax.set_ylim((0.45, 1.00))

    ax.set_title(f'Regression with Polynomial of Degree {lPolyDeg[ii]}')
    ax.set_xlabel('$\lambda$')
    ax.set_ylabel('${R}^{2}$')

    ax.legend()

* <font color='brown'>(**#**)</font> There is a regularization which use both norms (${L}_{1}$ and ${L}_{2}$) for regularization. It is called Elastic Net. See SciKit Learn's [`ElasticNet`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html).